In [21]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
import re

In [22]:
def make_lag(data, lags, week):
    for lag in range(1, lags + 1):
        if lag == 0 and week == 0:
            continue
        data[f"lag_{lag}_{week}"] = data.groupby("Store")["Sales"].shift(lag + 7 * week)
    return data

def clean_col_name(name):
    return re.sub(r"[(),=\s-]", "_", str(name))


In [23]:
def baseline_engineering(data):
    data = data.copy()
    data["Date"] = pd.to_datetime(data["Date"])
    data = data.sort_values(by=["Store", "Date"])
    data = data.set_index("Date")

    lag_frame = data[["Store", "Sales"]].copy()
    make_lag(lag_frame, 3, 0)
    make_lag(lag_frame, 1, 1)
    lag_frame["rolling_mean_7"] = (
        lag_frame.groupby("Store")["Sales"]
        .rolling(7, min_periods=1)
        .mean()
        .shift(1)
        .reset_index(level=0, drop=True)
    )
    lag_frame["rolling_std_7"] = (
        lag_frame.groupby("Store")["Sales"]
        .rolling(7, min_periods=1)
        .std()
        .shift(1)
        .fillna(0)
        .reset_index(level=0, drop=True)
    )

    baseline_features = data.copy()
    baseline_features = baseline_features.drop(
        [
            "Customers",
            "CompetitionDistance",
            "CompetitionOpenSinceMonth",
            "CompetitionOpenSinceYear",
            "Promo2",
            "Promo2SinceWeek",
            "Promo2SinceYear",
            "PromoInterval",
        ],
        axis=1,
    )
    baseline_features["day"] = (data.index - data.index.min()).days
    to_category = [
        "Open",
        "Promo",
        "StateHoliday",
        "SchoolHoliday",
        "StoreType",
        "Assortment",
    ]
    baseline_features[to_category] = baseline_features[to_category].astype("category")
    baseline_features["DayofMonth"] = data.index.day

    res = []
    for store_id, df_store in data.groupby("Store"):
        fouriers = CalendarFourier(freq="A", order=10)
        dp = DeterministicProcess(
            index=df_store.index,
            constant=True,
            order=1,
            seasonal=False,
            additional_terms=[fouriers],
            drop=True,
        )
        X = dp.in_sample()
        X["Store"] = store_id
        X["Date"] = df_store.index
        res.append(X)
    X_dp = pd.concat(res, ignore_index=True)

    baseline_features = baseline_features.merge(X_dp, on=["Store", "Date"], how="left")
    baseline_features = baseline_features.merge(lag_frame, on=["Date", "Store"], how="left")
    baseline_features = baseline_features.dropna()
    baseline_features = baseline_features[baseline_features["Date"] > "2013-01-08"]
    baseline_features = baseline_features[baseline_features["Open"] == 1]
    baseline_features = baseline_features.sort_values(by="Date")

    baseline_features.columns = [clean_col_name(c) for c in baseline_features.columns]
    return baseline_features, data["Sales"]


In [24]:
main=pd.read_csv('../data/train.csv')
sub=pd.read_csv('../data/store.csv')
data=pd.merge(main,sub,on='Store',how='left')
data['Date']=pd.to_datetime(data['Date'])
data.sort_values(by=['Store','Date'],inplace=True)
tmp=data.copy()
#data.set_index('Date',inplace=True)

/tmp/ipykernel_107589/1250609072.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  main=pd.read_csv('../data/train.csv')


In [25]:
data.index

Index([1016095, 1014980, 1013865, 1012750, 1011635, 1010520, 1009405, 1008290,
       1007175, 1006060,
       ...
         11149,   10034,    8919,    7804,    6689,    5574,    4459,    3344,
          2229,    1114],
      dtype='int64', length=1017209)

In [26]:
feature, y = baseline_engineering(data)
feature.head()


Index(['Store', 'Sales'], dtype='object')


/home/yjjjc/python/myv1/lib/python3.12/site-packages/statsmodels/tsa/deterministic.py:569: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  index = pd.date_range("2020-01-01", freq=freq, periods=1)
/home/yjjjc/python/myv1/lib/python3.12/site-packages/statsmodels/tsa/deterministic.py:569: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  index = pd.date_range("2020-01-01", freq=freq, periods=1)
/home/yjjjc/python/myv1/lib/python3.12/site-packages/statsmodels/tsa/deterministic.py:569: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  index = pd.date_range("2020-01-01", freq=freq, periods=1)
/home/yjjjc/python/myv1/lib/python3.12/site-packages/statsmodels/tsa/deterministic.py:569: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  index = pd.date_range("2020-01-01", freq=freq, periods=1)


,Store,Date,DayOfWeek,Sales_x,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,cos_9_freq_YE_DEC_,sin_10_freq_YE_DEC_,cos_10_freq_YE_DEC_,Sales_y,lag_1_0,lag_2_0,lag_3_0,lag_1_1,rolling_mean_7,rolling_std_7
8,1,2013-01-09,3,5471,1,1,0,1,c,a,...,0.325342,0.981306,0.192452,5471,5580.0,7176.0,0.0,0.0,4585.142857,2231.018633
115962,128,2013-01-09,3,5572,1,1,0,1,d,c,...,0.325342,0.981306,0.192452,5572,6465.0,7819.0,0.0,0.0,4945.428571,2475.318488
783816,861,2013-01-09,3,1559,1,1,0,0,c,c,...,0.325342,0.981306,0.192452,1559,1929.0,1711.0,0.0,0.0,1614.857143,745.003899
784758,862,2013-01-09,3,8640,1,1,0,0,a,c,...,0.325342,0.981306,0.192452,8640,10733.0,12571.0,0.0,0.0,7751.428571,4003.608949
115204,127,2013-01-09,3,6133,1,1,0,0,d,a,...,0.325342,0.981306,0.192452,6133,5431.0,7204.0,0.0,0.0,3892.714286,2273.777013


In [27]:
feature.columns

Index(['Store', 'Date', 'DayOfWeek', 'Sales_x', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment', 'day',
       'DayofMonth', 'const', 'trend', 'sin_1_freq_YE_DEC_',
       'cos_1_freq_YE_DEC_', 'sin_2_freq_YE_DEC_', 'cos_2_freq_YE_DEC_',
       'sin_3_freq_YE_DEC_', 'cos_3_freq_YE_DEC_', 'sin_4_freq_YE_DEC_',
       'cos_4_freq_YE_DEC_', 'sin_5_freq_YE_DEC_', 'cos_5_freq_YE_DEC_',
       'sin_6_freq_YE_DEC_', 'cos_6_freq_YE_DEC_', 'sin_7_freq_YE_DEC_',
       'cos_7_freq_YE_DEC_', 'sin_8_freq_YE_DEC_', 'cos_8_freq_YE_DEC_',
       'sin_9_freq_YE_DEC_', 'cos_9_freq_YE_DEC_', 'sin_10_freq_YE_DEC_',
       'cos_10_freq_YE_DEC_', 'Sales_y', 'lag_1_0', 'lag_2_0', 'lag_3_0',
       'lag_1_1', 'rolling_mean_7', 'rolling_std_7'],
      dtype='object')

In [28]:
data.columns

Index(['Store', 'DayOfWeek', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

In [29]:
# 加入更多周周期滞后信息
nf = pd.DataFrame()
nf[["Store", "Date"]] = tmp[["Store", "Date"]].copy()

nf["lag_14"] = tmp.groupby("Store")["Sales"].shift(14).copy()
nf["lag_21"] = tmp.groupby("Store")["Sales"].shift(21).copy()
nf["lag_28"] = tmp.groupby("Store")["Sales"].shift(28).copy()

nf["rolling_mean_28"] = tmp.groupby("Store")["Sales"].transform(
    lambda x: x.shift(1).rolling(28, center=False, min_periods=1).mean()
)
nf["rolling_std_28"] = tmp.groupby("Store")["Sales"].transform(
    lambda x: x.shift(1).rolling(28, center=False, min_periods=1).std()
)

nf = nf.dropna()
nf = nf.merge(feature, on=["Store", "Date"], how="inner")
nf.head()


,Store,Date,lag_14,lag_21,lag_28,rolling_mean_28,rolling_std_28,DayOfWeek,Sales_x,Open,...,cos_9_freq_YE_DEC_,sin_10_freq_YE_DEC_,cos_10_freq_YE_DEC_,Sales_y,lag_1_0,lag_2_0,lag_3_0,lag_1_1,rolling_mean_7,rolling_std_7
0,1,2013-01-29,3900.0,5580.0,0.0,4121.285714,2075.631146,2,3725,1,...,-0.365723,-0.994218,0.107381,3725,4055.0,0.0,5598.0,5394.0,4533.142857,2080.017823
1,1,2013-01-30,4008.0,5471.0,5530.0,4254.321429,1914.845455,3,4601,1,...,-0.217723,-0.961130,0.276097,4601,3725.0,4055.0,0.0,5720.0,4248.142857,2026.274696
2,1,2013-01-31,4044.0,4892.0,4327.0,4221.142857,1899.913267,4,4709,1,...,-0.064508,-0.899631,0.436651,4709,4601.0,3725.0,4055.0,5578.0,4108.571429,1951.681400
3,1,2013-02-01,4127.0,4881.0,4486.0,4234.785714,1902.071860,5,5633,1,...,0.090252,-0.811539,0.584298,5633,4709.0,4601.0,3725.0,5195.0,4039.142857,1914.889329
4,1,2013-02-02,5182.0,4952.0,4997.0,4275.750000,1919.949818,6,5970,1,...,0.242850,-0.699458,0.714673,5970,5633.0,4709.0,4601.0,5586.0,4045.857143,1921.288841


In [30]:
best_parms={
            'objective': 'regression',       # 回归问题
            'metric': 'rmse',                # 训练时优化 RMSE (对应 log_sales)
            'boosting_type': 'gbdt',
            'verbosity': -1,                 # 不打印乱七八糟的日志
            'n_jobs': -1,                    # 使用所有 CPU 核心
            'seed': 42,
            'learning_rate': 0.05801820833937896,
            'num_leaves': 144, 
            'max_depth': 13, 
            'min_child_samples': 18,
            'subsample': 0.9129847936149911,
            'colsample_bytree': 0.9384389823955829,
            'reg_alpha': 0.6588925840200691,
            'reg_lambda': 1.5944281549844126}

In [31]:
from sklearn.model_selection import TimeSeriesSplit
def train_lgbm_by_date(feature, y, params, k, date_col='day'):
    """
    基于整型日期列 (int) 的时间序列交叉验证
    
    Parameters
    ----------
    feature : pd.DataFrame
        特征矩阵，必须包含 date_col
    y : pd.Series
        目标变量 (注意：这里必须传入 log_Sales)
    params : dict
        LightGBM 参数
    k : int
        折数 (Folds)
    date_col : str
        日期列名，该列必须是 int 类型 (e.g., 1, 2, 3...)
    """

    # 1. 取出所有唯一日期（整型），并排序
    unique_dates = np.sort(feature[date_col].unique())
    rmspe_list = []
    
    # 2. 构造 fold (按时间切分)
    # array_split 会尽量均分，保证每一折的时间跨度大致相同
    date_folds = np.array_split(unique_dates, k + 1)
    
    print(f"开始时间序列交叉验证 (Total samples: {len(feature)})...")
    
    for fold_n in range(k):
        # 3. 定义日期区间
        # 训练集：从第0块到第n块
        # 验证集：第 n+1 块
        train_dates = np.concatenate(date_folds[:fold_n + 1])
        val_dates   = date_folds[fold_n + 1]
        
        # 4. 筛选数据
        train_mask = feature[date_col].isin(train_dates)
        val_mask   = feature[date_col].isin(val_dates)
        
        # 剔除日期列，避免模型直接把 "Day" 当作特征死记硬背
        # (虽然 Day 本身作为 int 有时也可以是特征，但通常作为 split 依据更好)
        X_train = feature.loc[train_mask].drop(columns=[date_col])
        X_val   = feature.loc[val_mask].drop(columns=[date_col])
        
        y_train = y.loc[train_mask]
        y_val   = y.loc[val_mask] # 这里的 y_val 是 log 后的值
        
        # 5. 构建 Dataset
        # 显式告诉 LGBM 自动识别 category 列
        lgb_train = lgb.Dataset(X_train, y_train, categorical_feature='auto')
        lgb_eval  = lgb.Dataset(X_val, y_val, reference=lgb_train, categorical_feature='auto')
        
        # 6. 训练
        gbm = lgb.train(
            params,
            lgb_train,
            num_boost_round=2000, 
            valid_sets=[lgb_train, lgb_eval],
            valid_names=['train', 'val'],
            callbacks=[
                lgb.early_stopping(stopping_rounds=50),
                lgb.log_evaluation(period=0) # 静默模式
            ]
        )
        
        # 7. 预测与评估 (关键还原步骤)
        # 预测出 Log Sales
        y_pred_log = gbm.predict(X_val, num_iteration=gbm.best_iteration)
        
        # 还原为 Real Sales
        y_true_real = np.expm1(y_val.values)
        y_pred_real = np.expm1(y_pred_log)
        
        # 计算 RMSPE (必须用还原后的值计算)
        score = rmspe(y_true_real, y_pred_real)
        rmspe_list.append(score)
        
        # 打印信息 (使用 int 日期显示范围)
        print(f"Fold {fold_n + 1} | RMSPE: {score:.5f}")
        print(f"  Train Days: {train_dates.min()} -> {train_dates.max()}")
        print(f"  Val   Days: {val_dates.min()} -> {val_dates.max()}")
        print("-" * 40)

    # 8. 汇总结果
    print(f"\nFinal CV RMSPE: {np.mean(rmspe_list):.5f} ± {np.std(rmspe_list):.5f}")
    
    return rmspe_list

In [32]:
ny=np.log1p(nf['Sales_x'])
nX=nf.drop(['Date','Sales_x','Sales_y'],axis=1)
nX.head()

,Store,lag_14,lag_21,lag_28,rolling_mean_28,rolling_std_28,DayOfWeek,Open,Promo,StateHoliday,...,sin_9_freq_YE_DEC_,cos_9_freq_YE_DEC_,sin_10_freq_YE_DEC_,cos_10_freq_YE_DEC_,lag_1_0,lag_2_0,lag_3_0,lag_1_1,rolling_mean_7,rolling_std_7
0,1,3900.0,5580.0,0.0,4121.285714,2075.631146,2,1,0,0,...,-0.930724,-0.365723,-0.994218,0.107381,4055.0,0.0,5598.0,5394.0,4533.142857,2080.017823
1,1,4008.0,5471.0,5530.0,4254.321429,1914.845455,3,1,0,0,...,-0.976011,-0.217723,-0.961130,0.276097,3725.0,4055.0,0.0,5720.0,4248.142857,2026.274696
2,1,4044.0,4892.0,4327.0,4221.142857,1899.913267,4,1,0,0,...,-0.997917,-0.064508,-0.899631,0.436651,4601.0,3725.0,4055.0,5578.0,4108.571429,1951.681400
3,1,4127.0,4881.0,4486.0,4234.785714,1902.071860,5,1,0,0,...,-0.995919,0.090252,-0.811539,0.584298,4709.0,4601.0,3725.0,5195.0,4039.142857,1914.889329
4,1,5182.0,4952.0,4997.0,4275.750000,1919.949818,6,1,0,0,...,-0.970064,0.242850,-0.699458,0.714673,5633.0,4709.0,4601.0,5586.0,4045.857143,1921.288841


In [33]:
nX[['Store','day']].head()

,Store,day
0,1,28
1,1,29
2,1,30
3,1,31
4,1,32


In [ ]:
train_lgbm_by_date(nX, ny,best_parms,5, date_col='day')

In [34]:
def iso_week_start(year, week):
    # ISO: 每周周一为开始
    return pd.to_datetime(f"{int(year)}-W{int(week):02d}-1", format="%G-W%V-%u")

# df: 已经有 Date, Promo2, Promo2SinceYear, Promo2SinceWeek
tmp["Promo2StartDate"] = tmp.apply(
    lambda r: iso_week_start(r["Promo2SinceYear"], r["Promo2SinceWeek"])
    if r["Promo2"] == 1
    and pd.notna(r["Promo2SinceYear"])
    and pd.notna(r["Promo2SinceWeek"])
    else pd.NaT,
    axis=1,
)

tmp["is_after_promo2_start"] = (tmp["Date"] >= tmp["Promo2StartDate"]).astype("int8")
tmp["is_promo2_active_raw"] = (
    (tmp["Promo2"] == 1) & (tmp["Date"] >= tmp["Promo2StartDate"])
).astype("int8")


In [35]:
month_map = {
    1: "Jan",
    2: "Feb",
    3: "Mar",
    4: "Apr",
    5: "May",
    6: "Jun",
    7: "Jul",
    8: "Aug",
    9: "Sept",
    10: "Oct",
    11: "Nov",
    12: "Dec",
}

tmp["is_promo_month"] = tmp.apply(
    lambda row: 1
    if month_map[row["Date"].month] in str(row["PromoInterval"])
    and row["is_promo2_active_raw"] == 1
    else 0,
    axis=1,
)
tmp["promo&promo2"] = tmp["is_promo_month"] * tmp["Promo"]


In [36]:
tmp["promo2_days_since_start"] = -1  # 默认：没有 Promo2
mask = tmp["Promo2"] == 1

tmp.loc[mask, "promo2_days_since_start"] = (
    tmp.loc[mask, "Date"] - tmp.loc[mask, "Promo2StartDate"]
).dt.days

# 已开始之前的 Promo2（理论上很少，但防御性处理）
tmp.loc[tmp["promo2_days_since_start"] < 0, "promo2_days_since_start"] = -1

tmp["promo2_weeks_since_start"] = (
    tmp["promo2_days_since_start"] // 7
).astype("int32")

tmp[
    [
        "Promo2StartDate",
        "is_after_promo2_start",
        "is_promo2_active_raw",
        "is_promo_month",
        "promo&promo2",
        "promo2_days_since_start",
        "promo2_weeks_since_start",
    ]
].head(10)


,Promo2StartDate,is_after_promo2_start,is_promo2_active_raw,is_promo_month,promo&promo2,promo2_days_since_start,promo2_weeks_since_start
1016095,NaT,0,0,0,0,-1,-1
1014980,NaT,0,0,0,0,-1,-1
1013865,NaT,0,0,0,0,-1,-1
1012750,NaT,0,0,0,0,-1,-1
1011635,NaT,0,0,0,0,-1,-1
1010520,NaT,0,0,0,0,-1,-1
1009405,NaT,0,0,0,0,-1,-1
1008290,NaT,0,0,0,0,-1,-1
1007175,NaT,0,0,0,0,-1,-1
1006060,NaT,0,0,0,0,-1,-1


In [37]:
nf = nf.merge(
    tmp[
        [
            "Store",
            "Date",
            "Promo2StartDate",
            "is_after_promo2_start",
            "is_promo2_active_raw",
            "is_promo_month",
            "promo&promo2",
            "promo2_days_since_start",
            "promo2_weeks_since_start",
        ]
    ],
    on=["Store", "Date"],
    how="left",
)
nf.head()


,Store,Date,lag_14,lag_21,lag_28,rolling_mean_28,rolling_std_28,DayOfWeek,Sales_x,Open,...,lag_1_1,rolling_mean_7,rolling_std_7,Promo2StartDate,is_after_promo2_start,is_promo2_active_raw,is_promo_month,promo&promo2,promo2_days_since_start,promo2_weeks_since_start
0,1,2013-01-29,3900.0,5580.0,0.0,4121.285714,2075.631146,2,3725,1,...,5394.0,4533.142857,2080.017823,NaT,0,0,0,0,-1,-1
1,1,2013-01-30,4008.0,5471.0,5530.0,4254.321429,1914.845455,3,4601,1,...,5720.0,4248.142857,2026.274696,NaT,0,0,0,0,-1,-1
2,1,2013-01-31,4044.0,4892.0,4327.0,4221.142857,1899.913267,4,4709,1,...,5578.0,4108.571429,1951.681400,NaT,0,0,0,0,-1,-1
3,1,2013-02-01,4127.0,4881.0,4486.0,4234.785714,1902.071860,5,5633,1,...,5195.0,4039.142857,1914.889329,NaT,0,0,0,0,-1,-1
4,1,2013-02-02,5182.0,4952.0,4997.0,4275.750000,1919.949818,6,5970,1,...,5586.0,4045.857143,1921.288841,NaT,0,0,0,0,-1,-1


In [ ]:
def add_interaction_features(df):
    df = df.copy()
    promo_flag = df["Promo"].astype("int8")
    school_flag = df["SchoolHoliday"].astype("int8")
    state_flag = (df["StateHoliday"].astype(str) != "0").astype("int8")
    promo2_flag = df["is_promo2_active_raw"].astype("int8")

    df["is_weekend"] = df["DayOfWeek"].isin([6, 7]).astype("int8")
    df["promo_weekend"] = (promo_flag * df["is_weekend"]).astype("int8")
    df["promo_schoolholiday"] = (promo_flag * school_flag).astype("int8")
    df["promo_stateholiday"] = (promo_flag * state_flag).astype("int8")
    df["promo2_weekend"] = (promo2_flag * df["is_weekend"]).astype("int8")

    df["storetype_assortment"] = (
        df["StoreType"].astype(str) + "_" + df["Assortment"].astype(str)
    ).astype("category")

    df["is_month_start"] = df["Date"].dt.is_month_start.astype("int8")
    df["is_month_end"] = df["Date"].dt.is_month_end.astype("int8")
    df["is_quarter_start"] = df["Date"].dt.is_quarter_start.astype("int8")
    df["is_quarter_end"] = df["Date"].dt.is_quarter_end.astype("int8")
    df["dayofyear"] = df["Date"].dt.dayofyear.astype("int16")
    return df

nf_interactions = add_interaction_features(nf)
nf_interactions.head()


In [38]:
ny = np.log1p(nf_interactions['Sales_x'])
nX = nf_interactions.drop(['Date', 'Sales_x', 'Sales_y'], axis=1)


In [39]:
nX.columns


Index(['Store', 'lag_14', 'lag_21', 'lag_28', 'rolling_mean_28',
       'rolling_std_28', 'DayOfWeek', 'Open', 'Promo', 'StateHoliday',
       'SchoolHoliday', 'StoreType', 'Assortment', 'day', 'DayofMonth',
       'const', 'trend', 'sin_1_freq_YE_DEC_', 'cos_1_freq_YE_DEC_',
       'sin_2_freq_YE_DEC_', 'cos_2_freq_YE_DEC_', 'sin_3_freq_YE_DEC_',
       'cos_3_freq_YE_DEC_', 'sin_4_freq_YE_DEC_', 'cos_4_freq_YE_DEC_',
       'sin_5_freq_YE_DEC_', 'cos_5_freq_YE_DEC_', 'sin_6_freq_YE_DEC_',
       'cos_6_freq_YE_DEC_', 'sin_7_freq_YE_DEC_', 'cos_7_freq_YE_DEC_',
       'sin_8_freq_YE_DEC_', 'cos_8_freq_YE_DEC_', 'sin_9_freq_YE_DEC_',
       'cos_9_freq_YE_DEC_', 'sin_10_freq_YE_DEC_', 'cos_10_freq_YE_DEC_',
       'lag_1_0', 'lag_2_0', 'lag_3_0', 'lag_1_1', 'rolling_mean_7',
       'rolling_std_7', 'Promo2StartDate', 'is_after_promo2_start',
       'is_promo2_active_raw', 'is_promo_month', 'promo&promo2',
       'promo2_days_since_start', 'promo2_weeks_since_start'],
      dtype='objec

In [41]:
tmp["Promo2StartDate"].dtype


dtype('<M8[ns]')

In [40]:
train_lgbm_by_date(nX, ny, best_parms, 5, date_col='day')


开始时间序列交叉验证 (Total samples: 818845)...


DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Float64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int8DType'>, <class 'numpy.dtypes.Int8DType'>, <class 'numpy.dtypes.Int8DType'>, <class 'numpy.dtypes.Int8DType'>, <class 'numpy.dtypes.Int8DType'>, <class 'numpy.dtypes.Int8DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dty